In [9]:
from openai import OpenAI
import os


client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

# First API call with reasoning
response = client.chat.completions.create(
  model="moonshotai/kimi-k2-thinking",
  messages=[
          {
            "role": "user",
            "content": "How many r's are in the word 'strawberry'?"
          }
        ]
)

# Extract the assistant message with reasoning_details
response = response.choices[0].message

# Preserve the assistant message with reasoning_details
messages = [
  {"role": "user", "content": "How many r's are in the word 'strawberry'?"},
  {
    "role": "assistant",
    "content": response.content,
    "reasoning_details": response.reasoning_details  # Pass back unmodified
  },
  {"role": "user", "content": "Are you sure? Think carefully."}
]

# Second API call - model continues reasoning from where it left off
response2 = client.chat.completions.create(
  model="moonshotai/kimi-k2-thinking",
  messages=messages,
  extra_body={"reasoning": {"enabled": True}}
)

In [6]:
response2

ChatCompletion(id='gen-1763992616-0Y6484v6MtVkotj5sSgO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You are right to ask me to double-check — it\'s a simple question that can be easy to miscount if you rush.\n\nLet me break it down **very carefully**:\n\nThe word is **s-t-r-a-w-b-e-r-r-y**.\n\nI\'ll go letter by letter and mark each \'r\':\n1. s\n2. t\n3. **r** ← (first r)\n4. a\n5. w\n6. b\n7. e\n8. **r** ← (second r)\n9. **r** ← (third r)\n10. y\n\nCounting them: **1, 2, 3**.\n\nThere are **3** r\'s in "strawberry".', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='The user is asking me to reconsider my answer to a very simple question. They want me to think carefully about how many r\'s are in "strawberry".\n\nLet me break down the word letter by letter:\n\nS - T - R - A - W - B - E - R - R - Y\n\nLet me index them:\n1. S\n2. T\n3. R\n4. A\n5. W\n6. B\n7. E\n8. R\n

In [10]:
response

ChatCompletionMessage(content=' There are **3** r\'s in the word "strawberry".', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='  The user is asking a simple counting question: "How many r\'s are in the word \'strawberry\'?"\n\nLet me spell out the word and count the \'r\'s:\n\nS-T-R-A-W-B-E-R-R-Y\n\nLet me go letter by letter:\n1. S\n2. T\n3. R (that\'s 1 r)\n4. A\n5. W\n6. B\n7. E\n8. R (that\'s 2 r\'s)\n9. R (that\'s 3 r\'s)\n10. Y\n\nSo there are 3 r\'s in the word "strawberry".\n\nI should answer this directly and clearly. ', reasoning_details=[{'format': 'unknown', 'index': 0, 'type': 'reasoning.text', 'text': '  The user is asking a simple counting question: "How many r\'s are in the word \'strawberry\'?"\n\nLet me spell out the word and count the \'r\'s:\n\nS-T-R-A-W-B-E-R-R-Y\n\nLet me go letter by letter:\n1. S\n2. T\n3. R (that\'s 1 r)\n4. A\n5. W\n6. B\n7. E\n8. R (that\'s 2 r\'s)\n9. R (that\'s 3 r\'s)\n10. Y\n

In [12]:
print(response.tool_calls)


None


In [2]:
import os
import json
import argparse
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from typing import Dict, Tuple, Union, List

import instructor
from instructor.core.client import Instructor
from instructor.core import InstructorRetryException, ValidationError as InstructorValidationError
import pandas as pd
from loguru import logger
from dotenv import load_dotenv
from openai.types.chat import ChatCompletion
from llama_index.llms.openai import OpenAI
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openrouter import OpenRouterProvider
from pydantic_ai.exceptions import UnexpectedModelBehavior
from pydantic_ai import Agent
from pydantic import ValidationError as PydanticValidationError
from pydantic_core import ValidationError as CoreValidationError

# UTILITY IMPORTS
from utils import ListOfEntities, PROMPT, validate_annotation_output_format, is_annotation_in_text

/home/touami/Bureau/mdner_llm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = os.getenv("OPENROUTER_API_KEY")
llm = OpenAIChatModel(
    "meta-llama/llama-3-70b-instruct",
    provider=OpenRouterProvider(api_key=api_key)
)
clients = llm

In [4]:
print(llm)

OpenAIChatModel()


In [5]:
agent = Agent(
    model=llm,
    output_type=ListOfEntities,
    retries=3,
    system_prompt=("Extract entities as structured JSON."),
)

In [8]:
try:
    response = await agent.run(f"{PROMPT}\nThe text to annotate:\n")
    print(response)
    result = response.output
    print(result)

except (PydanticValidationError, CoreValidationError, UnexpectedModelBehavior) as e:
    print(str(e))

ModelHTTPError: status_code: 404, model_name: meta-llama/llama-3-70b-instruct, body: {'message': "No endpoints found that support the provided 'tool_choice' value. To learn more about provider routing, visit: https://openrouter.ai/docs/provider-routing", 'code': 404}

In [28]:
agent = Agent(
    model=llm,
    output_type=ListOfEntities,
    retries=3,
    system_prompt=("Extract entities as structured JSON."),
)
try:
    response = await agent.run(f"{PROMPT}\nThe text to annotate:\n")
    print(response)
    result = response.output
    print(result)

except (PydanticValidationError, CoreValidationError, UnexpectedModelBehavior) as e:
    print(str(e))

AgentRunResult(output=ListOfEntities(entities=[Molecule(label='MOL', text='T cell receptor'), Molecule(label='MOL', text='T Cell Receptor'), Molecule(label='MOL', text='TCR'), Molecule(label='MOL', text='peptide-major histocompatibility complex'), Molecule(label='MOL', text='pMHC'), Molecule(label='MOL', text='TCRs'), Molecule(label='SOFTNAME', text='Modeller'), Molecule(label='SOFTNAME', text='ColabFold')]))
entities=[Molecule(label='MOL', text='T cell receptor'), Molecule(label='MOL', text='T Cell Receptor'), Molecule(label='MOL', text='TCR'), Molecule(label='MOL', text='peptide-major histocompatibility complex'), Molecule(label='MOL', text='pMHC'), Molecule(label='MOL', text='TCRs'), Molecule(label='SOFTNAME', text='Modeller'), Molecule(label='SOFTNAME', text='ColabFold')]


In [30]:
TEXT_TO_ANNOTATE = """Improved Coarse-Grained Modeling of Cholesterol-Containing Lipid Bilayers
Cholesterol trafficking, which is an essential function in mammalian cells, is intimately connected to molecular-scale interactions through cholesterol modulation of membrane structure and dynamics and interaction with membrane receptors. Since these effects of cholesterol occur on micro- to millisecond time scales, it is essential to develop accurate coarse-grained simulation models that can reach these time scales. Cholesterol has been shown experimentally to thicken the membrane and increase phospholipid tail order between 0 and 40% cholesterol, above which these effects plateau or slightly decrease. Here, we showed that the published MARTINI coarse-grained force-field for phospholipid (POPC) and cholesterol fails to capture these effects. Using reference atomistic simulations, we systematically modified POPC and cholesterol bonded parameters in MARTINI to improve its performance. We showed that the corrections to pseudobond angles between glycerol and the lipid tails and around the oleoyl double bond particle (the angle-corrected model ) slightly improves the agreement of MARTINI with experimentally measured thermal, elastic, and dynamic properties of POPC membranes. The angle-corrected model improves prediction of the thickening and ordering effects up to 40% cholesterol but overestimates these effects at higher cholesterol concentration. In accordance with prior work that showed the cholesterol rough face methyl groups are important for limiting cholesterol self-association, we revised the coarse-grained representation of these methyl groups to better match cholesterol-cholesterol radial distribution functions from atomistic simulations. In addition, by using a finer-grained representation of the branched cholesterol tail than MARTINI, we improved predictions of lipid tail order and bilayer thickness across a wide range of concentrations. Finally, transferability testing shows that a model incorporating our revised parameters into DOPC outperforms other CG models in a DOPC/cholesterol simulation series, which further argues for its efficacy and generalizability. These results argue for the importance of systematic optimization for coarse-graining biologically important molecules like cholesterol with complicated molecular structure.
"""

In [32]:
try:
    response = await agent.run(f"{PROMPT}\nThe text to annotate:\n {TEXT_TO_ANNOTATE}")
    print(response)
    result = response.output
    print(result)

except (PydanticValidationError, CoreValidationError, UnexpectedModelBehavior) as e:
    print(str(e))

AgentRunResult(output=ListOfEntities(entities=[Molecule(label='MOL', text='cholesterol'), Molecule(label='MOL', text='POPC'), Molecule(label='MOL', text='glycerol'), Molecule(label='MOL', text='DOPC'), Molecule(label='FFM', text='MARTINI')]))
entities=[Molecule(label='MOL', text='cholesterol'), Molecule(label='MOL', text='POPC'), Molecule(label='MOL', text='glycerol'), Molecule(label='MOL', text='DOPC'), Molecule(label='FFM', text='MARTINI')]
